<a href="https://colab.research.google.com/github/2pterons/twigfarm/blob/main/SST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chunbo777/Stable-Style-Transformer
!git clone https://github.com/lijuncen/Sentiment-and-Style-Transfer
!pip install transformers
!pip install sentencepiece
%cd /content/Stable-Style-Transformer/generation_model/yelp

Cloning into 'Stable-Style-Transformer'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 265 (delta 139), reused 249 (delta 123), pack-reused 0
Receiving objects: 100% (265/265), 1.54 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Cloning into 'Sentiment-and-Style-Transfer'...
remote: Enumerating objects: 392, done.
remote: Total 392 (delta 0), reused 0 (delta 0), pack-reused 392
Receiving objects: 100% (392/392), 61.34 MiB | 29.15 MiB/s, done.
Resolving deltas: 100% (149/149), done.
     |████████████████████████████████| 2.8 MB 8.6 MB/s 
     |████████████████████████████████| 895 kB 76.4 MB/s 
     |████████████████████████████████| 3.3 MB 55.1 MB/s 
     |████████████████████████████████| 50 kB 8.2 MB/s 
     |████████████████████████████████| 636 kB 64.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML

In [2]:
pip install tensorboardX

     |████████████████████████████████| 124 kB 7.6 MB/s 


In [3]:
cd /content/Stable-Style-Transformer/generation_model/yelp/classifier

/content/Stable-Style-Transformer/generation_model/yelp/classifier


In [ ]:
import torch
import torch.nn as nn
import numpy as np

from tqdm import tqdm
import os
import random

from transformers import *
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
from tqdm import tqdm
import json


## 초기화
from dis_model import *
dismodel = findattribute().cuda()
dismodel.train()

import torch.optim as optim

from tensorboardX import SummaryWriter
summary = SummaryWriter(logdir='./logs')

def main():    
    f = open('../gpt_yelp_vocab.json')
    token2num = json.load(f)

    num2token = {}
    for key, value in token2num.items():
        num2token[value] = key
    f.close()

    data_path = "/content/Sentiment-and-Style-Transfer/data" # customize data path
    yelp_neg_path = data_path + "/yelp/sentiment.train.0"
    yelp_neg_open = open(yelp_neg_path, "r")
    yelp_neg_dataset = yelp_neg_open.readlines()
    neg_len = len(yelp_neg_dataset)
    yelp_neg_open.close()

    yelp_pos_path = data_path + "/yelp/sentiment.train.1"
    yelp_pos_open = open(yelp_pos_path, "r")
    yelp_pos_dataset = yelp_pos_open.readlines()
    pos_len = len(yelp_pos_dataset)
    yelp_pos_open.close()

    """training parameter"""
    cls_initial_lr = 0.001
    cls_trainer = optim.Adamax(dismodel.cls_params, lr=cls_initial_lr) # initial 0.001
    max_grad_norm = 25
    batch = 1
    epoch = 5
    stop_point = pos_len*epoch
    
    pre_epoch = 0
    for start in tqdm(range(0, stop_point)):
        ## learing rate decay
        now_epoch = (start+1)//pos_len
        if now_epoch == 4:
            cls_initial_lr = cls_initial_lr/2            
            cls_trainer = optim.Adamax(dismodel.cls_params, lr=cls_initial_lr) # initial 0.001
            
        """data start point"""
        neg_start = start%neg_len
        pos_start = start%pos_len

        """data setting"""
        neg_sentence = yelp_neg_dataset[neg_start].strip()
        pos_sentence = yelp_pos_dataset[pos_start].strip()                

        neg_labels = [] # negative labels
        neg_labels.append([1,0])
        neg_attribute = torch.from_numpy(np.asarray(neg_labels)).type(torch.FloatTensor).cuda()

        pos_labels = [] # positive labels
        pos_labels.append([0,1])
        pos_attribute = torch.from_numpy(np.asarray(pos_labels)).type(torch.FloatTensor).cuda()

        sentences = [neg_sentence, pos_sentence]
        attributes = [neg_attribute, pos_attribute]

        """data input"""
        for i in range(2):
            # k=0: negative, k=1: positive
            sentence = sentences[i]
            attribute = attributes[i] # for generate

            token_idx = torch.tensor(gpt_tokenizer.encode(sentence)).unsqueeze(0).cuda()
            
            dis_out = dismodel.discriminator(token_idx)

            """calculation loss & traning"""
            # training using discriminator loss
            cls_loss = dismodel.cls_loss(attribute, dis_out)
            summary.add_scalar('discriminator loss', cls_loss.item(), start)

            cls_trainer.zero_grad()
            cls_loss.backward() # retain_graph=True
            grad_norm = torch.nn.utils.clip_grad_norm_(dismodel.cls_params, max_grad_norm)            
            cls_trainer.step()
        
        """savining point"""
        if (start+1)%pos_len == 0:
            random.shuffle(yelp_neg_dataset)
            random.shuffle(yelp_pos_dataset)
            save_model((start+1)//pos_len)        
    save_model('final') # final_model    

    
def save_model(iter):
    if not os.path.exists('/content/drive/MyDrive/models2/'):
        os.makedirs('/content/drive/MyDrive/models2/')
    torch.save(dismodel.state_dict(), '/content/drive/MyDrive/models2/cls_model_{}'.format(iter))  
    

if __name__ == '__main__':
    torch.cuda.empty_cache()
    main()

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

  0%|          | 0/1330205 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 4386/1330205 [00:49<4:06:00, 89.82it/s]

In [ ]:
%pip install torch

In [ ]:
#cd ..

In [ ]:
#gen_model, train.py
import torch
import torch.nn as nn
import numpy as np

from tqdm import tqdm
import os
import random

from transformers import *
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
from tqdm import tqdm
import json


## 초기화
from gen_model import *
genmodel = styletransfer().cuda()
genmodel.train()

sys.path.insert(0, "/content/Stable-Style-Transformer/generation_model/yelp/classifier")
from dis_model import *
dismodel = findattribute().cuda()
dismodel_name='cls_model_3'
dismodel.load_state_dict(torch.load('/content/drive/MyDrive/models2/{}'.format(dismodel_name)))
dismodel.eval()


import torch.optim as optim

from tensorboardX import SummaryWriter
summary = SummaryWriter(logdir='./logs')

def main():    
    f = open('/content/Stable-Style-Transformer/generation_model/yelp/gpt_yelp_vocab.json')
    token2num = json.load(f)

    num2token = {}
    for key, value in token2num.items():
        num2token[value] = key
    f.close()

    data_path = ""
    train_yelp_neg_path = data_path + "/content/Sentiment-and-Style-Transfer/data/yelp/sentiment.train.0"
    train_yelp_neg_open = open(train_yelp_neg_path, "r")
    train_yelp_neg_dataset = train_yelp_neg_open.readlines()
    yelp_neg_dataset = train_yelp_neg_dataset
    
    neg_len = len(yelp_neg_dataset)
    train_yelp_neg_open.close()

    train_yelp_pos_path = data_path + "/content/Sentiment-and-Style-Transfer/data/yelp/sentiment.train.1"
    train_yelp_pos_open = open(train_yelp_pos_path, "r")
    train_yelp_pos_dataset = train_yelp_pos_open.readlines()
    yelp_pos_dataset = train_yelp_pos_dataset
    
    pos_len = len(yelp_pos_dataset)
    train_yelp_pos_open.close()

    """training parameter"""
    aed_initial_lr = 0.00001
    gen_initial_lr = 0.001
    aed_trainer = optim.Adamax(genmodel.aed_params, lr=aed_initial_lr) # initial 0.0005
    gen_trainer = optim.Adamax(genmodel.aed_params, lr=gen_initial_lr) # initial 0.0001
    max_grad_norm = 20
    batch = 1
    epoch = 6
    stop_point = pos_len*epoch
    
    pre_epoch = 0
    for start in tqdm(range(0, stop_point)):
        ## learing rate decay
        now_epoch = (start+1)//pos_len
            
        """data start point"""
        neg_start = start%neg_len
        pos_start = start%pos_len

        """data setting"""
        neg_sentence = yelp_neg_dataset[neg_start].strip()
        pos_sentence = yelp_pos_dataset[pos_start].strip()                

        neg_labels = [] # negative labels
        neg_labels.append([1,0])
        neg_attribute = torch.from_numpy(np.asarray(neg_labels)).type(torch.FloatTensor).cuda()

        pos_labels = [] # positive labels
        pos_labels.append([0,1])
        pos_attribute = torch.from_numpy(np.asarray(pos_labels)).type(torch.FloatTensor).cuda()

        sentences = [neg_sentence, pos_sentence]
        attributes = [neg_attribute, pos_attribute]
        sentiments = [0, 1]

        """data input"""
        for i in range(2):
            # k=0: negative, k=1: positive
            sentence = sentences[i]
            attribute = attributes[i] # for decoder
            fake_attribute = attributes[abs(1-i)] # for generate
#             sentiment = sentiments[i] # for delete

            token_idx = torch.tensor(gpt_tokenizer.encode(sentence)).unsqueeze(0).cuda()

            # delete model
            max_len = int(token_idx.shape[1]/2)
            dis_out = dismodel.discriminator(token_idx)    
            sentiment = dis_out.argmax(1).cpu().item() ## 변경점 for delete
            
            del_idx = token_idx
            for k in range(max_len):
                del_idx = dismodel.att_prob(del_idx, sentiment)                
                dis_out = dismodel.discriminator(del_idx)    
                sent_porb = F.softmax(dis_out, 1).squeeze(0)[sentiment].cpu().detach().numpy().item()
                if sent_porb < 0.7:
                    break       
                    
            """auto-encoder loss & traning"""
            # training using discriminator loss
            enc_out = genmodel.encoder(del_idx)
            dec_out, vocab_out = genmodel.decoder(enc_out, token_idx, attribute)

            ## calculation loss
            recon_loss = genmodel.recon_loss(token_idx, vocab_out)
            summary.add_scalar('reconstruction loss', recon_loss.item(), start)
            
            aed_trainer.zero_grad()
            recon_loss.backward(retain_graph=True) # retain_graph=True
            grad_norm = torch.nn.utils.clip_grad_norm_(genmodel.aed_params, max_grad_norm)            
            aed_trainer.step()
            
            """decoder classification loss & training"""
            ## calculation loss
            gen_cls_out = dismodel.gen_discriminator(vocab_out)

            ## calculation loss
            gen_cls_loss = genmodel.cls_loss(attribute, gen_cls_out)
            summary.add_scalar('generated sentence loss', gen_cls_loss.item(), start)

            gen_trainer.zero_grad()
            gen_cls_loss.backward() # retain_graph=True
            grad_norm = torch.nn.utils.clip_grad_norm_(genmodel.aed_params, max_grad_norm)
            gen_trainer.step()
            
        
        """savining point"""
        if (start+1)%pos_len == 0:
            random.shuffle(yelp_neg_dataset)
            random.shuffle(yelp_pos_dataset)
            save_model((start+1)//pos_len)        
    save_model('final') # final_model    

    
def save_model(iter):
    if not os.path.exists('/content/drive/MyDrive/model_gen'):
        os.makedirs('/content/drive/MyDrive/model_gen/')
    torch.save(genmodel.state_dict(), '/content/drive/MyDrive/model_gen/gen_model_{}'.format(iter))  
    

if __name__ == '__main__':
    torch.cuda.empty_cache()
    main()

In [ ]:
pwd

In [ ]:
#yelp.classifier의 dis_model

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

sys.path.insert(0, "/DATA/joosung/fairseq_master")

class findattribute(nn.Module):
    def __init__(self, drop_rate=0, gpu = True):
        super(findattribute, self).__init__()
        self.gpu = gpu
        
        self.n_vocab = 50259
        self.emb_dim = 256
        
        """idx & length"""
        self.START_IDX = 50257
        self.PAD_IDX = 50258
        self.EOS_IDX = 50256
        
        """Discriminator(classifier)"""
        self.word_dim = 256
        self.word_emb = nn.Embedding(self.n_vocab, self.word_dim, self.PAD_IDX) # 50265x1024
        
        self.channel_out = 100
        self.conv2d_2 = nn.Conv2d(1,self.channel_out,(2,self.word_dim))
        self.conv2d_3 = nn.Conv2d(1,self.channel_out,(3,self.word_dim))
        self.conv2d_4 = nn.Conv2d(1,self.channel_out,(4,self.word_dim))
        self.conv2d_5 = nn.Conv2d(1,self.channel_out,(5,self.word_dim))
#         self.fc_drop = nn.Dropout(drop_rate)
        self.disc_fc = nn.Linear(4*self.channel_out, 2)
        
        """parameters"""                
        self.cls_params = list(self.word_emb.parameters())+list(self.conv2d_2.parameters())+list(self.conv2d_3.parameters())+list(self.conv2d_4.parameters())+\
        list(self.conv2d_5.parameters())+list(self.disc_fc.parameters())
            

    def discriminator(self, token_idx):
        """
        token_idx: (batch, seq_len)
        """
        if token_idx.shape[1] < 5:
            padding_size = 5-token_idx.shape[1]
            padding_token = []
            for k in range(token_idx.shape[0]):
                temp = []
                for i in range(padding_size):
                    temp.append(self.PAD_IDX)
                padding_token.append(temp)                
            padding_token=torch.from_numpy(np.array(padding_token))
            if self.gpu == True:
                padding_token = padding_token.cuda()
            token_idx=torch.cat([token_idx,padding_token], 1) # (batch, seq_len+padding) = (batch, 5)

        word_emb = self.word_emb(token_idx) # (batch, seq_len, word_dim)
        word_2d = word_emb.unsqueeze(1) # (batch, 1, seq_len, word_dim)

        x2 = F.relu(self.conv2d_2(word_2d)).squeeze(3) # bi-gram, (batch, channel_out, seq_len-1)
        x3 = F.relu(self.conv2d_3(word_2d)).squeeze(3) # 3-gram, (batch, channel_out, seq_len-2)
        x4 = F.relu(self.conv2d_4(word_2d)).squeeze(3) # 4-gram, (batch, channel_out, seq_len-3)
        x5 = F.relu(self.conv2d_5(word_2d)).squeeze(3) # 5-gram, (batch, channel_out, seq_len-4)

        # Max-over-time-pool
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2) # (batch, channel_out)
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2) # (batch, channel_out)
        x4 = F.max_pool1d(x4, x4.size(2)).squeeze(2) # (batch, channel_out)
        x5 = F.max_pool1d(x5, x5.size(2)).squeeze(2) # (batch, channel_out)
        x = torch.cat([x2, x3, x4, x5], dim=1) # (batch, channel_out*4)

        y = self.disc_fc(x) # (batch, 2)

        if self.gpu == True:
            return y.cuda()
        else:
            return y
        
    def gen_discriminator(self, gen_out):
        """
        gen_out: (gen_len+2, batch, n_vocab)
        """
        gen_emb = gen_out[1:-1,:,:] # (gen_len, batch, n_vocab)
        gen_emb = torch.bmm(gen_emb, self.word_emb.weight.repeat(gen_emb.shape[0],1,1))
        # (gen_len, batch, emb_dim) = (gen_len, batch, n_vocab) x (gen_len, n_vocab, emb_dim)
        gen_emb = gen_emb.transpose(0, 1) # (batch, gen_len, word_dim)
        
        if gen_emb.shape[1] < 5:
            padding_size = 5-gen_emb.shape[1]
            padding_token = []
            for k in range(gen_emb.shape[0]):
                temp = []
                for i in range(padding_size):
                    temp.append(self.PAD_IDX)
                padding_token.append(temp)                
            padding_token=torch.from_numpy(np.array(padding_token)) # (batch, padding_len)
            if self.gpu == True:
                padding_token = padding_token.cuda()
            padding_emb = self.word_emb(padding_token) # (batch, padding_len, emb_dim)
            gen_emb = torch.cat([gen_emb, padding_emb], 1) # (batch, 5, emb_dim)   
            
        word_2d = gen_emb.unsqueeze(1) # (batch, 1, seq_len, word_dim)

        x2 = F.relu(self.conv2d_2(word_2d)).squeeze(3) # bi-gram, (batch, channel_out, seq_len-1)
        x3 = F.relu(self.conv2d_3(word_2d)).squeeze(3) # 3-gram, (batch, channel_out, seq_len-2)
        x4 = F.relu(self.conv2d_4(word_2d)).squeeze(3) # 4-gram, (batch, channel_out, seq_len-3)
        x5 = F.relu(self.conv2d_5(word_2d)).squeeze(3) # 5-gram, (batch, channel_out, seq_len-4)

        # Max-over-time-pool
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2) # (batch, channel_out)
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2) # (batch, channel_out)
        x4 = F.max_pool1d(x4, x4.size(2)).squeeze(2) # (batch, channel_out)
        x5 = F.max_pool1d(x5, x5.size(2)).squeeze(2) # (batch, channel_out)
        x = torch.cat([x2, x3, x4, x5], dim=1) # (batch, channel_out*4)

        y = self.disc_fc(x) # (batch, 2)

        if self.gpu == True:
            return y.cuda()
        else:
            return y
        
    def att_prob(self, token_idx, sentiment):
        """
        token_idx: (batch, seq_len)
        """
#         if token_idx.size(1) < 5:
#             padding_size = 5-token_idx.size(1)
#             padding_token = []
#             for k in range(token_idx.size(0)):
#                 temp = []
#                 for i in range(padding_size):
#                     temp.append(self.PAD_IDX)
#                 padding_token.append(temp)                
#             padding_token=torch.from_numpy(np.array(padding_token))
#             if self.gpu == True:
#                 padding_token = padding_token.cuda()
#             token_idx=torch.cat([token_idx,padding_token], 1) # (batch, seq_len+padding) = (batch, 5)
        token_list = token_idx.squeeze(0).cpu().tolist() # list
        min_prob = 1
        for i in range(len(token_list)):
            del_list = token_list[:i] + token_list[i+1:]
            del_tensor = torch.from_numpy(np.asarray(del_list)).unsqueeze(0).cuda()
            del_prob=F.softmax(self.discriminator(del_tensor),1).squeeze(0)[sentiment].cpu().detach().numpy().item()
            
            if del_prob <= min_prob:                
                max_ind = i
                min_prob = del_prob
                
        final_list = token_list[:max_ind] + token_list[max_ind+1:]
        del_idx = torch.from_numpy(np.asarray(final_list)).unsqueeze(0).cuda()
        return del_idx    
        
    def cls_loss(self, targets, cls_out):
        """
        targets: (batch, 2) / attributes [0,1] or [1,0]
        cls_out: (batch, 2) (logits)
        """
        
        final_targets = targets.argmax(1) # (batch)
        cls_loss = F.cross_entropy(cls_out, final_targets)
        
        if self.gpu == True:       
            return cls_loss.cuda()
        else:
            return cls_loss